<a href="https://colab.research.google.com/github/jude1997felix/Decentralized-Air-quality-Monitoring-and-Prediction/blob/main/competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.mlab as mlab
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
import os
import datetime
import math 
import IPython
import IPython.display
import matplotlib as mpl
import tensorflow as tf

In [86]:
def compile_and_fit(model, window, patience=5):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history



In [87]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds


def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels




# Data

In [303]:
df = pd.read_csv('/content/imputed_csv_18644.csv',parse_dates=['Start'])
df.head()

,Unnamed: 0,Start,Slut,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,0,2015-01-01 00:00:00,2015-01-01 01:00:00,11.0,18.8,27.9,19.8
1,1,2015-01-01 01:00:00,2015-01-01 02:00:00,15.3,25.0,15.5,4.5
2,2,2015-01-01 02:00:00,2015-01-01 03:00:00,13.4,19.8,17.1,4.6
3,3,2015-01-01 03:00:00,2015-01-01 04:00:00,9.6,13.8,15.2,5.3
4,4,2015-01-01 04:00:00,2015-01-01 05:00:00,3.5,5.8,16.3,6.6


In [304]:
del df['Unnamed: 0']
# del df['Black Carbon (410)']

In [305]:
df_copy = df.copy()
df.head()

,Start,Slut,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,2015-01-01 00:00:00,2015-01-01 01:00:00,11.0,18.8,27.9,19.8
1,2015-01-01 01:00:00,2015-01-01 02:00:00,15.3,25.0,15.5,4.5
2,2015-01-01 02:00:00,2015-01-01 03:00:00,13.4,19.8,17.1,4.6
3,2015-01-01 03:00:00,2015-01-01 04:00:00,9.6,13.8,15.2,5.3
4,2015-01-01 04:00:00,2015-01-01 05:00:00,3.5,5.8,16.3,6.6


# Model

## Data splitting and windowing


In [306]:
  del df['Start']
  del df['Slut']
  column_indices = {name: i for i, name in enumerate(df.columns)}

  n = len(df)

  cols = df.columns
  x = df.values #returns a numpy array
  sc = StandardScaler()
  x_scaled = sc.fit_transform(x)
  df = pd.DataFrame(x_scaled)

  # y = pm10_test.values
  # y_scaled = sc.fit_transform(y)
  # pm10_test = 
  
  df.columns = cols
  
  # Normalising training data
  train_df = df[0:int(n*0.8)]
  # x = train_df.values
  # train_scaled = sc.fit_transform(x)
  # train_df = pd.DataFrame(train_scaled,columns=cols)
  
  # Normalising validation data
  val_df = df[int(n*0.80):int(n*0.90)]
  # x = val_df.values
  # train_scaled = sc.fit_transform(x)
  # val_df = pd.DataFrame(train_scaled,columns=cols)
  
  # Normalising testing data
  test_df = df[41000:41100]
  # x = test_df.values
  # train_scaled = sc.fit_transform(x)
  # test_df = pd.DataFrame(train_scaled,columns=cols)


  num_features = df.shape[1]
  # return(train_df,val_df,test_df,num_features)
  train_mean = train_df.mean()
  train_std = train_df.std()

  df_std = (df - train_mean) / train_std

In [307]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
                train_df=train_df, val_df=val_df, test_df=test_df,
                label_columns=None):
      # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

      # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                      enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}

      # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])


WindowGenerator.split_window = split_window
# WindowGenerator.plot = plot
WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
      # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
      # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

  

# single_step_window = WindowGenerator(
#       input_width=2, label_width=1, shift=1,
#       label_columns=['PM10 (107)'])
#   # single_step_window
# print(single_step_window.test)


In [308]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

# multi_window.plot()
multi_window

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label column name(s): None

## Model training and prediction using DNN

In [309]:
MAX_EPOCHS = 20
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

# IPython.display.clear_output()
# multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
# multi_performance = multi_dense_model.predict(multi_window.test)
# multi_window.plot(multi_dense_model)

Epoch 1/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8287 - mean_absolute_error: 0.6338 - val_loss: 0.8698 - val_mean_absolute_error: 0.6639
Epoch 2/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8140 - mean_absolute_error: 0.6264 - val_loss: 0.8662 - val_mean_absolute_error: 0.6658
Epoch 3/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8103 - mean_absolute_error: 0.6248 - val_loss: 0.8661 - val_mean_absolute_error: 0.6618
Epoch 4/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8077 - mean_absolute_error: 0.6237 - val_loss: 0.8652 - val_mean_absolute_error: 0.6646
Epoch 5/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8064 - mean_absolute_error: 0.6232 - val_loss: 0.8627 - val_mean_absolute_error: 0.6619
Epoch 6/20
1095/1095 [==============================] - 5s 5ms/step - loss: 0.8049 - mean_absolute_error: 0.6226 - val_loss: 0.8629 - val_mean_absolute_error: 0.6629
Epoc

In [310]:
predictions = dict()
for i in range(92):
  idx = 41591  + i*24
  test_df = df.iloc[idx : idx+48]
  
  class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                  train_df=train_df, val_df=val_df, test_df=test_df,
                  label_columns=None):
        # Store the raw data.
      self.train_df = train_df
      self.val_df = val_df
      self.test_df = test_df

        # Work out the label column indices.
      self.label_columns = label_columns
      if label_columns is not None:
        self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
      self.column_indices = {name: i for i, name in
                              enumerate(train_df.columns)}

        # Work out the window parameters.
      self.input_width = input_width
      self.label_width = label_width
      self.shift = shift

      self.total_window_size = input_width + shift

      self.input_slice = slice(0, input_width)
      self.input_indices = np.arange(self.total_window_size)[self.input_slice]

      self.label_start = self.total_window_size - self.label_width
      self.labels_slice = slice(self.label_start, None)
      self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
      return '\n'.join([
          f'Total window size: {self.total_window_size}',
          f'Input indices: {self.input_indices}',
          f'Label indices: {self.label_indices}',
          f'Label column name(s): {self.label_columns}'])


  WindowGenerator.split_window = split_window
  # WindowGenerator.plot = plot
  WindowGenerator.make_dataset = make_dataset

  @property
  def train(self):
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
        # And cache it for next time
      self._example = result
    return result

  WindowGenerator.train = train
  WindowGenerator.val = val
  WindowGenerator.test = test
  WindowGenerator.example = example


  OUT_STEPS = 24
  multi_window = WindowGenerator(input_width=24,
                                label_width=OUT_STEPS,
                                shift=OUT_STEPS)

  # multi_window.plot()
  # multi_window


  pred = multi_dense_model.predict(multi_window.test)
  pred = pred.reshape(24,4)
  predictions[i] = pred
idx

43775

In [311]:
test_df.shape

(48, 4)

## Denormalizing and calculating the smape values

In [312]:
idx = 41615
for days in range(92):
  for hour in range(24):
    for feat in range(4):
      if feat == 0:
        df['NO2 (344)'].iloc[idx] = predictions[days][hour][feat]
      elif feat == 1:
        df['NOX as NO2 (345)'].iloc[idx] = predictions[days][hour][feat]
      elif feat == 2:
        df['PM10 (342)'].iloc[idx] = predictions[days][hour][feat]
      else:
        df['PM2.5 (343)'].iloc[idx] = predictions[days][hour][feat]
    idx = idx+1


In [313]:
x = df.values
x_scaled = sc.inverse_transform(x)
df_norm = pd.DataFrame(x_scaled, columns=cols)
df_norm.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,11.0,18.8,27.9,19.8
1,15.3,25.0,15.5,4.5
2,13.4,19.8,17.1,4.6
3,9.6,13.8,15.2,5.3
4,3.5,5.8,16.3,6.6


In [314]:
del df_copy['Slut']
del df_copy['Start']
df_copy.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
0,11.0,18.8,27.9,19.8
1,15.3,25.0,15.5,4.5
2,13.4,19.8,17.1,4.6
3,9.6,13.8,15.2,5.3
4,3.5,5.8,16.3,6.6


In [315]:
y_test = df_copy[41615:]
y_pred = df_norm[41615:]

In [316]:
y_test.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
41615,4.3,9.7,0.0,0.002842
41616,4.7,7.7,2.0,0.000000
41617,1.8,3.2,2.3,0.000000
41618,1.3,1.9,2.2,0.053142
41619,3.5,6.0,3.8,0.400000


In [317]:
y_pred.head()

,NO2 (344),NOX as NO2 (345),PM10 (342),PM2.5 (343)
41615,19.471270,60.659505,5.046160,0.296953
41616,24.767853,79.357072,7.662711,1.024119
41617,27.597073,88.657137,9.913626,1.251994
41618,29.104562,91.542055,10.836983,1.530252
41619,31.074538,96.405039,12.291671,1.982532


In [318]:
def smape(y_true, y_pred):
  return tf.reduce_mean(2 * tf.abs(y_true - y_pred)
                          / (tf.abs(y_pred) + tf.abs(y_true)) , axis=-1)


In [319]:
smape(y_test['NO2 (344)'],y_pred['NO2 (344)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.480165756426967>

In [320]:
smape(y_test['NOX as NO2 (345)'],y_pred['NOX as NO2 (345)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.5882156345836959>

In [321]:
smape(y_test['PM10 (342)'],y_pred['PM10 (342)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.55922429258736>

In [322]:
smape(y_test['PM2.5 (343)'],y_pred['PM2.5 (343)'])

<tf.Tensor: shape=(), dtype=float64, numpy=0.5594407440002999>

In [326]:
(0.4801 + 0.5882 + 0.5592 + 0.5594)/4

0.546725

In [327]:
y_pred.to_csv("test_values_18644.csv")

In [325]:
0.4886 + 0.5954 + 0.5534 + 0.5669

0.4801 + 0.5882 + 0.5592 + 0.5594

2.2043